In [ ]:
pip install impyute

In [ ]:
pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

In [ ]:
test = pd.read_csv('/content/drive/My Drive/data/kaggle/house-prices/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/data/kaggle/house-prices/train.csv', encoding = 'cp949')


#데이터 타입 확인

In [ ]:
train.info()

#NULL 개수

In [ ]:
train.isnull().sum()

#명목형 변수

In [ ]:
category_feature = [ col for col in train.columns if train[col].dtypes == "object"]
category_feature


# 다중대처법(컬럼삭제 x)

In [ ]:
train_no_salep_price = train.drop(['SalePrice'], axis=1)


In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
total = pd.concat([train_no_salep_price,test],axis=0,ignore_index=True)
total

In [ ]:
import missingno as msno
msno.matrix(total)


In [ ]:
dummy = pd.get_dummies(total)

In [ ]:
dummy

In [ ]:
total_impute = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy))
total_impute

In [ ]:
msno.matrix(total_impute)


In [ ]:
total_cols = list(dummy.columns)
total_impute.columns = total_cols
# pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy)).head()


In [ ]:
X_train = total_impute[:1460] 
X_test = total_impute[1460:] 
# y_test = test['SalePrice']
# train['SalePrice'] = np.log1p(train['SalePrice'])

In [ ]:
y_train = np.log1p(train['SalePrice'])


In [ ]:
total_impute.shape

In [ ]:
msno.bar(total_impute)

# 공산성

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
pd.options.display.float_format = '{:.5f}'.format


수치형 변수

In [ ]:
Numeric_feature = [ col for col in train.columns if train[col].dtypes != "object"]
Numeric_feature

VIF

In [ ]:
y, X = dmatrices('SalePrice ~ Id+MSSubClass+LotFrontage+LotArea+OverallQual+OverallCond+YearBuilt+YearRemodAdd+MasVnrArea+BsmtFinSF1+BsmtFinSF2+BsmtUnfSF+TotalBsmtSF++LowQualFinSF+GrLivArea+BsmtFullBath+BsmtHalfBath+FullBath+HalfBath+BedroomAbvGr+KitchenAbvGr+TotRmsAbvGrd+Fireplaces+GarageYrBlt+GarageCars+GarageArea+WoodDeckSF+OpenPorchSF+EnclosedPorch+ScreenPorch+PoolArea+MiscVal+MoSold+YrSold+SalePrice', train, return_type = 'dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

범주형 변수

In [ ]:
import statsmodels.api as sm;
import scipy.stats as stats
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_cor = train[category_feature]
train_cor

In [ ]:
corr = train.corr(method='spearman')
corr

In [ ]:
sns.heatmap(corr, cmap='viridis')
plt.show()

In [ ]:
len(corr.columns)

In [ ]:
# 수치형은 그런걸 뺴긴 해야됨
check_list = []
for i in corr:
    count = 0 
    for j in corr[i]:
        if j != 1.0 and j>0.7:
            print(i,corr.columns[count],j)
            check_list.append([i,corr.columns[count],j])
        count+=1


In [ ]:
# 빼야될 항목 GarageArea , GarageYrBlt, TotRmsAbvGrd , 1stFlrSF (saleprice와 상관관계가 낮음)
stats.spearmanr(train['SalePrice'],train['1stFlrSF'])

# 주성분분석(PCA)



In [ ]:
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 
pca_train = pd.get_dummies(train)
x = df_train.values # 독립변인들의 value값만 추출
y = train['SalePrice'].values # 종속변인 추출
x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = df_train.columns
pd.DataFrame(x, columns=features).head()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=printcipalComponents)
# 주성분으로 이루어진 데이터 프레임 구성

In [ ]:
principalDf.head()


In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)


# 요인분석

In [ ]:
pip install factor_analyzer

In [ ]:
from factor_analyzer import FactorAnalyzer

In [ ]:
items = df_train.columns

In [ ]:
fa = FactorAnalyzer(n_factors=8, method="ml", rotation="promax")
fa.fit(df_train)

In [ ]:
fa.loadings_

In [ ]:
efa_result= pd.DataFrame(fa.loadings_, index=items)

In [ ]:
plt.figure(figsize=(6,10))
sns.heatmap(efa_result, cmap="Blues", annot=True, fmt='.2f')

In [ ]:
def CronbachAlpha(itemscores):
    itemscores = np.asarray(itemscores)
    itemvars = itemscores.var(axis=0, ddof=1)
    tscores = itemscores.sum(axis=1)
    nitems = itemscores.shape[1]
    return (nitems / (nitems-1)) * (1 - (itemvars.sum() / tscores.var(ddof=1)))


# 머신러닝

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics

In [ ]:
# train['SalePrice'] = np.log1p(train['SalePrice'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=777)


In [ ]:
print(len(X_test),len(y_test))

## Random Forest

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## XG boost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

## DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## LassoCV

In [ ]:
from sklearn.linear_model import LassoCV
model = LassoCV()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## ElasticNetCV

In [ ]:
from sklearn.linear_model import ElasticNetCV
model = ElasticNetCV()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## SVR

In [ ]:
from sklearn.svm import LinearSVR
model = LinearSVR()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## BayesianRidge

In [ ]:
from sklearn.linear_model import BayesianRidge
model = BayesianRidge()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

## LGBM

In [ ]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Root Mean Squared Error:',rmse(y_pred,y_test))

# 앙상블 기법

## voting

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

# ensemble 할 model 정의
models = [
    # ('enc', ElasticNetCV()),
    # ('bc', LinearSVR()),
    ('br',BayesianRidge()),
    # ('rfc', RandomForestRegressor()),
    # ('knn', KNeighborsRegressor()),
    # ('lc', LassoCV()),
    # ('lgbm', LGBMRegressor()),
    # ('dtr', DecisionTreeRegressor()),
    ('xgb', XGBRegressor()),
    ('gbr',GradientBoostingRegressor()),
    # ('rf', RandomForestRegressor())
]
voting_regressor = VotingRegressor(models, n_jobs=-1)
voting_regressor.fit(X_train, y_train)
voting_pred = voting_regressor.predict(X_test)
print('Root Mean Squared Error:',rmse(voting_pred,y_test))

## stacking

In [ ]:
from sklearn.ensemble import StackingRegressor
models = [
    # ('enc', ElasticNetCV()),
    # ('lsvr', LinearSVR()),
    ('br',BayesianRidge()),
    # ('rfc', RandomForestRegressor()),
    ('knn', KNeighborsRegressor()),
    # ('lc', LassoCV()),
    ('lgbm', LGBMRegressor()),
    # ('dtr', DecisionTreeRegressor()),
    ('xgb', XGBRegressor()),
    ('gbr',GradientBoostingRegressor()),
    # ('rf', RandomForestRegressor()),

]
xgb = XGBRegressor()
gbr = GradientBoostingRegressor()
lgbm = LGBMRegressor()
br = BayesianRidge()
rf = RandomForestRegressor()
stack_reg = StackingRegressor(models, final_estimator=gbr, n_jobs=-1)
stack_reg.fit(X_train, y_train)
stack_pred = stack_reg.predict(X_test)
print('Root Mean Squared Error:',rmse(stack_pred,y_test))